In [27]:
import pandas as pd, os
from utils.util import is_laptop
from datetime import date

In [2]:
# For laptop
hourly_path = 'D:/Skóli/lokaverkefni_vel/data/Vedurstofa/combined_klst.feather'
ten_minute_path = 'D:/Skóli/lokaverkefni_vel/data/merged_measured_reanalysis_landscape_2sectors_25ms_24hr_25_3_24.feather'

In [9]:
# For PC
folder_path = 'E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/' if not is_laptop() else None
hourly_path = folder_path + 'Measured/combined_klst/' + max(os.listdir(folder_path + 'Measured/combined_klst/'), key = lambda f: os.path.getmtime(folder_path + 'Measured/combined_klst/' + f))
merged_path = folder_path + 'Combined/' + max(os.listdir(folder_path + 'Combined/'), key = lambda f: os.path.getmtime(folder_path + 'Combined/' + f))

In [10]:
hourly_df = pd.read_feather(hourly_path)
merged_df = pd.read_feather(merged_path)

In [13]:
# Rounds to next hour so as to be able to directly compare with the vedur klst file
def next_hour(time):
    return time.ceil('h')

In [14]:
hourly_df.stod = pd.to_numeric(hourly_df.stod, errors = 'coerce')
hourly_df.timi = pd.to_datetime(hourly_df.timi, errors = 'coerce')
hourly_df.fx = pd.to_numeric(hourly_df.fx, errors = 'coerce')

merged_df['timi'] = merged_df.time.apply(next_hour)

In [15]:
merged_df = pd.merge(merged_df, hourly_df, on = ['stod', 'timi'], how = 'inner', suffixes=('_merged', '_hourly'))

In [20]:
nailStripped_df = merged_df[abs(merged_df.f_merged - merged_df.fx) <= 0.1]

In [25]:
df_toWrite = nailStripped_df[['X', 'Y', 'time', 'ws_15', 'ws_250', 'ws_500', 'wd_15',
       'wd_250', 'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500',
       'stod', 'f_merged', 'fg_merged', 'd_merged', 'Ri_01', 'Ri_12', 'Ri_02', 'N_01', 'N_12', 'N_02']]

df_toWrite.columns

Index(['X', 'Y', 'time', 'ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250',
       'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'stod',
       'f_merged', 'fg_merged', 'd_merged', 'Ri_01', 'Ri_12', 'Ri_02', 'N_01',
       'N_12', 'N_02'],
      dtype='object')

In [26]:
df_toWrite = df_toWrite.rename({'f_merged':'f', 'fg_merged':'fg', 'd_merged':'d'}, axis = 1)

df_toWrite.columns

Index(['X', 'Y', 'time', 'ws_15', 'ws_250', 'ws_500', 'wd_15', 'wd_250',
       'wd_500', 'p_15', 'p_250', 'p_500', 't_15', 't_250', 't_500', 'stod',
       'f', 'fg', 'd', 'Ri_01', 'Ri_12', 'Ri_02', 'N_01', 'N_12', 'N_02'],
      dtype='object')

In [29]:
outputpath = folder_path + f'NailStripped/merged_stripped_{date.today()}.feather'
df_toWrite.to_feather(outputpath)